**IMPORT LIBRARIES**

In [1]:
import psycopg2 as ps
import numpy as np
import pandas as pd

**CONNECT TO DATABASE**

In [2]:
host_name = 'localhost'
dbname = 'owid_covid_19_db'
port = '5432'
username = 'postgres' 
password = 'password'
conn = None

In [3]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [4]:
conn = connect_to_db(host_name, dbname,port, username, password)

Connected!


In [5]:
curr = conn.cursor()

**UPLOAD DATA**

In [6]:
df_1 = pd.read_excel('DATA/covid-testing-latest-data-source-details.xlsx')
df_2 = pd.read_csv('DATA/covid-variants.csv')
df_3 = pd.read_csv('DATA/excess-deaths.csv')
df_4 = pd.read_csv('DATA/owid-covid-data.csv')
df_5 = pd.read_csv('DATA/vaccine-type data RSA.csv', sep = '\t')

In [7]:
df_1.replace([np.inf, -np.inf], np.nan, inplace=True)
df_2.replace([np.inf, -np.inf], np.nan, inplace=True)
df_3.replace([np.inf, -np.inf], np.nan, inplace=True)
df_4.replace([np.inf, -np.inf], np.nan, inplace=True)
df_5.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
def replace_null_with_zero(cols):
    value = cols
    if pd.isnull(value):
        return 0
    else:
        return value

In [9]:
for df in [df_1,df_2,df_3,df_4,df_5]:
    for col in df.columns:
      df[col] = df[col].apply(replace_null_with_zero)

In [10]:
def make_zero(num):
    if num < 0:
        return 0
    else:
        return num

In [11]:
df_4['new_deaths'] = df_4['new_deaths'].apply(lambda x : make_zero(x))

**DELETE VIEWS AND TABLE**

In [12]:
def delete_view_1(curr):
    delete_view_command = ("""DROP VIEW IF EXISTS main_covid_19_data""")
    curr.execute(delete_view_command)

In [13]:
def delete_view_2(curr):
    delete_view_command = ("""DROP VIEW IF EXISTS continent_population""")
    curr.execute(delete_view_command)

In [14]:
def delete_view_3(curr):
    delete_view_command = ("""DROP VIEW IF EXISTS country_population""")
    curr.execute(delete_view_command)

In [15]:
delete_view_1(curr)
delete_view_2(curr)
delete_view_3(curr)
conn.commit()

In [16]:
def delete_table_1(curr):
    delete_table_command = ("""DROP TABLE IF EXISTS covid_testing""")
    curr.execute(delete_table_command)

In [17]:
def delete_table_2(curr):
    delete_table_command = ("""DROP TABLE IF EXISTS covid_variants""")
    curr.execute(delete_table_command)

In [18]:
def delete_table_3(curr):
    delete_table_command = ("""DROP TABLE IF EXISTS excess_deaths""")
    curr.execute(delete_table_command)

In [19]:
def delete_table_4(curr):
    delete_table_command = ("""DROP TABLE IF EXISTS owid_covid_table""")
    curr.execute(delete_table_command)

In [20]:
def delete_table_5(curr):
    delete_table_command = ("""DROP TABLE IF EXISTS vaccine_type""")
    curr.execute(delete_table_command)

In [21]:
delete_table_1(curr)
delete_table_2(curr)
delete_table_3(curr)
delete_table_4(curr)
delete_table_5(curr)
conn.commit()

**CREATE TABLES**

In [22]:
def create_table_1(curr):
    create_table_command = ("""CREATE TABLE covid_testing(
                ISO_code VARCHAR(50),
                Entity VARCHAR(150),
                Date VARCHAR(100),
                Source_URL VARCHAR(500),
                Source_label VARCHAR(150),
                Notes VARCHAR(150),
                Number_of_observations INTEGER,
                Cumulative_total NUMERIC,
                Cumulative_total_per_thousand NUMERIC,
                Daily_change_in_cumulative_total NUMERIC,
                Daily_change_in_cumulative_total_per_thousand NUMERIC,
                seven_day_smoothed_daily_change NUMERIC,
                seven_day_smoothed_daily_change_per_thousand NUMERIC,
                Short_term_positive_rate NUMERIC,
                Short_term_tests_per_case NUMERIC,
                General_source_label VARCHAR(200),
                General_source_URL VARCHAR(500),
                Short_description VARCHAR(500)
                )""")
    curr.execute(create_table_command)

In [23]:
def create_table_2(curr):
    create_table_command = ("""CREATE TABLE covid_variants(
                location VARCHAR(150),
                date DATE,
                varient VARCHAR(150),
                num_sequences NUMERIC,
                perc_sequences NUMERIC,
                num_sequences_total NUMERIC
                )""")
    curr.execute(create_table_command)

In [24]:
def create_table_3(curr):
    create_table_command = ("""CREATE TABLE excess_deaths(
                location VARCHAR(150),
                date DATE,
                p_scores_all_ages NUMERIC,
                p_scores_15_64 NUMERIC,
                p_scores_65_74 NUMERIC,
                p_scores_75_84 NUMERIC,
                p_scores_85plus NUMERIC,
                deaths_2020_all_ages NUMERIC,
                average_deaths_2015_2019_all_ages NUMERIC,
                deaths_2015_all_ages NUMERIC,
                deaths_2016_all_ages NUMERIC,
                deaths_2017_all_ages NUMERIC,
                deaths_2018_all_ages NUMERIC,
                deaths_2019_all_ages NUMERIC,
                deaths_2010_all_ages NUMERIC,
                deaths_2011_all_ages NUMERIC,
                deaths_2012_all_ages NUMERIC,
                deaths_2013_all_ages NUMERIC,
                deaths_2014_all_ages NUMERIC,
                deaths_2021_all_ages NUMERIC,
                time NUMERIC,
                time_unit VARCHAR(100),
                p_scores_0_14 NUMERIC,
                projected_deaths_2020_all_ages NUMERIC,
                excess_proj_all_ages NUMERIC,
                cum_excess_proj_all_ages NUMERIC,
                cum_proj_deaths_all_ages NUMERIC,
                cum_p_proj_all_ages NUMERIC,
                p_proj_all_ages NUMERIC,
                p_proj_0_14 NUMERIC,
                p_proj_15_64 NUMERIC,
                p_proj_65_74 NUMERIC,
                p_proj_75_84 NUMERIC,
                p_proj_85p NUMERIC,
                cum_excess_per_million_proj_all_ages NUMERIC
                )""")
    curr.execute(create_table_command)

In [25]:
def create_table_4(curr):
    create_table_command = ("""CREATE TABLE owid_covid_table(
                iso_code VARCHAR(50),
                continent VARCHAR(50),
                location VARCHAR(150),
                date DATE,
                total_cases NUMERIC,
                new_cases NUMERIC,
                new_cases_smoothed NUMERIC,
                total_deaths NUMERIC,
                new_deaths NUMERIC,
                new_deaths_smoothed NUMERIC,
                total_cases_per_million NUMERIC,
                new_cases_per_million NUMERIC,
                new_cases_smoothed_per_million NUMERIC,
                total_deaths_per_million NUMERIC,
                new_deaths_per_million NUMERIC,
                new_deaths_smoothed_per_million NUMERIC,
                reproduction_rate NUMERIC,
                icu_patients NUMERIC,
                icu_patients_per_million NUMERIC,
                hosp_patients NUMERIC,
                hosp_patients_per_million NUMERIC,
                weekly_icu_admissions NUMERIC,
                weekly_icu_admissions_per_million NUMERIC,
                weekly_hosp_admissions NUMERIC,
                weekly_hosp_admissions_per_million NUMERIC,
                new_tests NUMERIC,
                total_tests NUMERIC,
                total_tests_per_thousand NUMERIC,
                new_tests_per_thousand NUMERIC,
                new_tests_smoothed NUMERIC,
                new_tests_smoothed_per_thousand NUMERIC,
                positive_rate NUMERIC,
                tests_per_case NUMERIC,
                tests_units VARCHAR(150),
                total_vaccinations NUMERIC,
                people_vaccinated NUMERIC,
                people_fully_vaccinated NUMERIC,
                total_boosters NUMERIC,
                new_vaccinations NUMERIC,
                new_vaccinations_smoothed NUMERIC,
                total_vaccinations_per_hundred NUMERIC,
                people_vaccinated_per_hundred NUMERIC,
                people_fully_vaccinated_per_hundred NUMERIC,
                total_boosters_per_hundred NUMERIC,
                new_vaccinations_smoothed_per_million NUMERIC,
                stringency_index NUMERIC,
                population NUMERIC,
                population_density NUMERIC,
                median_age NUMERIC,
                aged_65_older NUMERIC,
                aged_70_older NUMERIC,
                gdp_per_capita NUMERIC,
                extreme_poverty NUMERIC,
                cardiovasc_death_rate NUMERIC,
                diabetes_prevalence NUMERIC,
                female_smokers NUMERIC,
                male_smokers NUMERIC,
                handwashing_facilities NUMERIC,
                hospital_beds_per_thousand NUMERIC,
                life_expectancy NUMERIC,
                human_development_index NUMERIC,
                excess_mortality_cumulative_absolute NUMERIC,
                excess_mortality_cumulative NUMERIC,
                excess_mortality NUMERIC,
                excess_mortality_cumulative_per_million NUMERIC
                )""")
    curr.execute(create_table_command)

In [26]:
def create_table_5(curr):
    create_table_command = ("""CREATE TABLE vaccine_type(
                location VARCHAR(150),
                date DATE,
                vaccine VARCHAR(200),
                source VARCHAR(300),
                total_vaccinations NUMERIC,
                people_vaccinated NUMERIC,
                people_fully_vaccinated NUMERIC,
                total_boosters NUMERIC
                )""")
    curr.execute(create_table_command)

In [27]:
create_table_1(curr)
create_table_2(curr)
create_table_3(curr)
create_table_4(curr)
create_table_5(curr)
conn.commit()

**INSERT INTO DATABASE**

**INSERT INTO TABLE 1**

In [28]:
def append_from_df_1_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table_1(curr,row['ISO code'], row['Entity'], row['Date'], row['Source URL'], row['Source label'], 
                        row['Notes'],row['Number of observations'], row['Cumulative total'],
                        row['Cumulative total per thousand'], row['Daily change in cumulative total'],
                        row['Daily change in cumulative total per thousand'], row['7-day smoothed daily change'],
                        row['7-day smoothed daily change per thousand'], row['Short-term positive rate'],
                        row['Short-term tests per case'], row['General source label'], row['General source URL'],
                        row['Short description'])

In [29]:
def insert_into_table_1(curr, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r):
    insert_into_covid_testing = ("""INSERT INTO covid_testing (ISO_code, Entity, Date, Source_URL, Source_label, Notes, 
    Number_of_observations, Cumulative_total, Cumulative_total_per_thousand, Daily_change_in_cumulative_total, 
    Daily_change_in_cumulative_total_per_thousand, seven_day_smoothed_daily_change, 
    seven_day_smoothed_daily_change_per_thousand, Short_term_positive_rate, Short_term_tests_per_case, 
    General_source_label, General_source_URL, Short_description)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ;""")
    row_to_insert = (a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r)
    curr.execute(insert_into_covid_testing, row_to_insert)

**INSERT INTO TABLE 2**

In [30]:
def append_from_df_2_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table_2(curr, row['location'], row['date'], row['variant'], row['num_sequences'], 
                        row['perc_sequences'], row['num_sequences_total'])

In [31]:
def insert_into_table_2(curr, a, b, c, d, e, f):
    insert_into_table_2 = ("""INSERT INTO covid_variants (location, date, varient, num_sequences, 
    perc_sequences, num_sequences_total)
    VALUES(%s,%s,%s,%s,%s,%s)
    ;""")
    row_to_insert = (a, b, c, d, e, f)
    curr.execute(insert_into_table_2, row_to_insert)

**INSERT INTO TABLE 3**

In [32]:
def append_from_df_3_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table_3(curr, row['location'], row['date'],row['p_scores_all_ages'], row['p_scores_15_64'],
                        row['p_scores_65_74'],row['p_scores_75_84'],row['p_scores_85plus'],
                        row['deaths_2020_all_ages'],row['average_deaths_2015_2019_all_ages'],
                        row['deaths_2015_all_ages'],row['deaths_2016_all_ages'],row['deaths_2017_all_ages'],
                        row['deaths_2018_all_ages'],row['deaths_2019_all_ages'],row['deaths_2010_all_ages'],
                        row['deaths_2011_all_ages'], row['deaths_2012_all_ages'], row['deaths_2013_all_ages'],
                        row['deaths_2014_all_ages'], row['deaths_2021_all_ages'], row['time'], row['time_unit'], 
                        row['p_scores_0_14'], row['projected_deaths_2020_all_ages'],
                        row['excess_proj_all_ages'], row['cum_excess_proj_all_ages'],
                        row['cum_proj_deaths_all_ages'], row['cum_p_proj_all_ages'], row['p_proj_all_ages'],
                        row['p_proj_0_14'], row['p_proj_15_64'], row['p_proj_65_74'], row['p_proj_75_84'],
                        row['p_proj_85p'], row['cum_excess_per_million_proj_all_ages'])

In [33]:
def insert_into_table_3(curr, a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,ab,ac,ad,ae,af,ag,ah,ai):
    insert_into_table_3 = ("""INSERT INTO excess_deaths (location, date, p_scores_all_ages, p_scores_15_64, p_scores_65_74,
    p_scores_75_84, p_scores_85plus, deaths_2020_all_ages, average_deaths_2015_2019_all_ages, deaths_2015_all_ages,
    deaths_2016_all_ages, deaths_2017_all_ages, deaths_2018_all_ages, deaths_2019_all_ages, deaths_2010_all_ages,
    deaths_2011_all_ages, deaths_2012_all_ages, deaths_2013_all_ages, deaths_2014_all_ages,deaths_2021_all_ages,time,time_unit,
    p_scores_0_14, projected_deaths_2020_all_ages, excess_proj_all_ages, cum_excess_proj_all_ages, cum_proj_deaths_all_ages,
    cum_p_proj_all_ages, p_proj_all_ages, p_proj_0_14, p_proj_15_64,
    p_proj_65_74, p_proj_75_84, p_proj_85p, cum_excess_per_million_proj_all_ages)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ;""")
    row_to_insert = (a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,ab,ac,ad,ae,af,ag,ah,ai)
    curr.execute(insert_into_table_3, row_to_insert)

**INSERT INTO TABEL 4**

In [34]:
def append_from_df_4_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table_4(curr, row['iso_code'], row['continent'], row['location'], row['date'], row['total_cases'], 
                        row['new_cases'],row['new_cases_smoothed'], row['total_deaths'], row['new_deaths'],
                       row['new_deaths_smoothed'], row['total_cases_per_million'],
                       row['new_cases_per_million'], row['new_cases_smoothed_per_million'],
                       row['total_deaths_per_million'], row['new_deaths_per_million'],
                       row['new_deaths_smoothed_per_million'], row['reproduction_rate'], row['icu_patients'],
                       row['icu_patients_per_million'], row['hosp_patients'],
                       row['hosp_patients_per_million'], row['weekly_icu_admissions'],
                       row['weekly_icu_admissions_per_million'], row['weekly_hosp_admissions'],
                       row['weekly_hosp_admissions_per_million'], row['new_tests'], row['total_tests'],
                       row['total_tests_per_thousand'], row['new_tests_per_thousand'],
                       row['new_tests_smoothed'], row['new_tests_smoothed_per_thousand'],
                       row['positive_rate'], row['tests_per_case'], row['tests_units'], row['total_vaccinations'],
                       row['people_vaccinated'], row['people_fully_vaccinated'], row['total_boosters'],
                       row['new_vaccinations'], row['new_vaccinations_smoothed'],
                       row['total_vaccinations_per_hundred'], row['people_vaccinated_per_hundred'],
                       row['people_fully_vaccinated_per_hundred'], row['total_boosters_per_hundred'],
                       row['new_vaccinations_smoothed_per_million'], row['stringency_index'],
                       row['population'], row['population_density'], row['median_age'], row['aged_65_older'],
                       row['aged_70_older'], row['gdp_per_capita'], row['extreme_poverty'],
                       row['cardiovasc_death_rate'], row['diabetes_prevalence'], row['female_smokers'],
                       row['male_smokers'], row['handwashing_facilities'], row['hospital_beds_per_thousand'],
                       row['life_expectancy'], row['human_development_index'],
                       row['excess_mortality_cumulative_absolute'], row['excess_mortality_cumulative'],
                       row['excess_mortality'], row['excess_mortality_cumulative_per_million'])

In [35]:
def insert_into_table_4(curr, a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,ab,ac,ad,ae,af,ag,
                        ah,ai,aj,ak,al,am,an,ao,ap,aq,ar,at,au,av,aw,ax,ay,az,ba,bb,bc,bd,be,bf,bg,bh,bj,bk,bl,bm,bn,bo):
    insert_into_table_4 = ("""INSERT INTO owid_covid_table (iso_code,
    continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,
    new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,
    total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,
    icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,
    weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,
    new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,
    new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,
    people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,
    people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,
    new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,
    aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,
    female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,
    excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,
    excess_mortality_cumulative_per_million)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ;""")
    row_to_insert = (a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,ab,ac,ad,ae,af,ag,
                        ah,ai,aj,ak,al,am,an,ao,ap,aq,ar,at,au,av,aw,ax,ay,az,ba,bb,bc,bd,be,bf,bg,bh,bj,bk,bl,bm,bn,bo)
    curr.execute(insert_into_table_4, row_to_insert)

**INSERT INTO TABLE 5**

In [36]:
def append_from_df_5_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table_5(curr, row['location'], row['date'], row['source_url'], row['total_vaccinations'],
                        row['people_vaccinated'], row['people_fully_vaccinated'], row['total_boosters'])

In [37]:
def insert_into_table_5(curr, a, b, c, d, e, f, g):
    insert_into_track_5 = ("""INSERT INTO vaccine_type (location,date,source,total_vaccinations,people_vaccinated,
    people_fully_vaccinated,total_boosters)
    VALUES(%s,%s,%s,%s,%s,%s,%s)
    ;""")
    row_to_insert = (a, b, c, d, e, f, g)
    curr.execute(insert_into_track_5, row_to_insert)

In [38]:
append_from_df_1_to_db(curr, df_1)
append_from_df_2_to_db(curr, df_2)
append_from_df_3_to_db(curr, df_3)
append_from_df_4_to_db(curr, df_4)
append_from_df_5_to_db(curr, df_5)
conn.commit()

**CREATING VIEWS**

In [39]:
def create_view_main_covid_19_data(curr):
    create_view_command = ("""CREATE VIEW main_covid_19_data AS
                SELECT continent, location, date, new_cases, new_deaths, new_tests, new_vaccinations, 
                people_vaccinated, people_fully_vaccinated, total_boosters 
                FROM owid_covid_table
                WHERE continent != '0'
                ORDER BY 2,3
                """)
    curr.execute(create_view_command)

In [40]:
def create_view_continent_population(curr):
    create_view_command = ("""CREATE VIEW continent_population AS 
                WITH new_table AS (SELECT location, 
                continent, MAX(population) AS population, MAX(population_density) AS population_density, 
                MAX(median_age) AS median_age, MAX(aged_65_older) AS aged_65_older, 
                MAX(aged_70_older) AS aged_70_older, MAX(gdp_per_capita) AS gdp_per_capita, 
                MAX(handwashing_facilities) AS handwashing_facilities, MAX(life_expectancy) AS life_expectancy
                FROM owid_covid_table
                WHERE continent != '0'
                GROUP BY continent,location
                ORDER BY 1) 

                SELECT continent, SUM(population) AS population, AVG(population_density) AS populaion_density, 
                AVG(median_age) AS median_age, AVG(aged_65_older) AS aged_65_older, 
                AVG(aged_70_older) AS aged_70_older, AVG(gdp_per_capita) AS gdp_per_capita, 
                SUM(handwashing_facilities) AS handwashing_facilities, AVG(life_expectancy) AS life_expectancy
                FROM new_table
                GROUP BY continent
                ORDER BY 1""")
    curr.execute(create_view_command)

In [41]:
def create_view_country_population(curr):
    create_view_command = ("""CREATE VIEW country_population AS
                SELECT location, MAX(population) AS population, MAX(population_density) AS populaion_density, 
                MAX(median_age) AS median_age, MAX(aged_65_older) AS aged_65_older, 
                MAX(aged_70_older) AS aged_70_older, MAX(gdp_per_capita) AS gdp_per_capita, 
                MAX(handwashing_facilities) AS handwashing_facities, MAX(life_expectancy) AS life_expenctancy
                FROM owid_covid_table
                WHERE continent != '0'
                GROUP BY location
                ORDER BY 1
                """)
    curr.execute(create_view_command)

In [42]:
delete_view_1(curr)
create_view_main_covid_19_data(curr)
delete_view_2(curr)
create_view_continent_population(curr)
delete_view_3(curr)
create_view_country_population(curr)
conn.commit()

14 Minute Run time